<a href="https://colab.research.google.com/github/neuralguy18/demo/blob/main/Pytorch_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
print(torch.__version__)

2.5.1+cu121


In [5]:
torch.cuda.is_available()

False

In [6]:
tensor0d = torch.tensor(1)

tensor1d = torch.tensor([1,2,3])

tensor2d = torch.tensor([[1,2],[3,4]])

tensor3d = torch.tensor([[[1,2],[3,4]],[[5,6],[7,8]]])

In [7]:
print(tensor1d.dtype)

torch.int64


In [8]:
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

torch.float32


In [9]:
print(tensor2d.shape)

torch.Size([2, 2])


In [10]:
print(tensor2d)

tensor([[1, 2],
        [3, 4]])


In [11]:
print(tensor2d.T)

tensor([[1, 3],
        [2, 4]])


In [12]:
print(tensor2d.matmul(tensor2d.T))

tensor([[ 5, 11],
        [11, 25]])


In [13]:
import torch.nn.functional as F
y = torch.tensor([1.0])
x1= torch.tensor([1.1])
w1= torch.tensor([2.2])
b= torch.tensor([0.0])

a = torch.sigmoid(x1*w1+b)
loss = F.binary_cross_entropy(a,y)
print(loss)

tensor(0.0852)


In [14]:
class NeuralNetwork(torch.nn.Module):
  def __init__(self,num_inputs,num_outputs):
    super().__init__()
    self.layers = torch.nn.Sequential(
        torch.nn.Linear(num_inputs,30),
        torch.nn.ReLU(),
        torch.nn.Linear(30,20),
        torch.nn.ReLU(),
        torch.nn.Linear(20,num_outputs)
    )

  def forward(self,x):
    logits = self.layers(x)
    return logits

In [15]:
model = NeuralNetwork(50,3)

In [16]:
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [17]:
model.parameters()

<generator object Module.parameters at 0x7e6a0c585690>

In [18]:
num_params = sum(p.numel() for p in model.parameters()
                    if p.requires_grad)
print(num_params)

2213


In [19]:
print(model.layers[2].weight.shape)

torch.Size([20, 30])


In [20]:
torch.manual_seed(123)
model = NeuralNetwork(50,3)
print(model.layers[4].weight)

Parameter containing:
tensor([[-0.1692, -0.0858,  0.2063, -0.1134, -0.1537,  0.0388,  0.1798, -0.0861,
         -0.0695, -0.0770, -0.1165,  0.0031, -0.0258,  0.0932, -0.1665, -0.0642,
         -0.2058, -0.2228,  0.0619, -0.0289],
        [-0.2011, -0.0067,  0.0681,  0.1624,  0.1344, -0.2128,  0.0601, -0.2104,
          0.1657,  0.1458, -0.2062,  0.0016,  0.0697,  0.0092, -0.2019,  0.2094,
         -0.1101,  0.0695,  0.1398, -0.2045],
        [-0.0663, -0.1034,  0.2191, -0.0906, -0.1899, -0.2023, -0.0230,  0.1957,
         -0.1776, -0.1351, -0.2014,  0.2151,  0.1206,  0.0938,  0.1765,  0.0676,
          0.1493, -0.0508,  0.0346, -0.1222]], requires_grad=True)


In [21]:
torch.manual_seed(123)
X = torch.rand((1,50))
with torch.no_grad():
  out = torch.softmax(model(X),dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


In [22]:
X_train = torch.tensor([
    [-1.2, 3.1,1.2],
    [-0.9, 2.9,0.7],
    [-0.5, 2.6,-0.8],
    [2.3, -1.1,-1.1],
    [2.7, -1.5,-1.2]
])
y_train = torch.tensor([0, 0, 0, 1, 1])

In [23]:
X_test = torch.tensor([
    [-0.8, 2.8,3.2],
    [2.6, -1.6,-2.0]
])
y_test = torch.tensor([0, 1])

In [24]:
from torch.utils.data import Dataset, DataLoader

class ToyDataset(Dataset):
    def __init__(self,X,y):
      self.features = X
      self.labels = y

    def __getitem__(self,index):
      one_x = self.features[index]
      one_y = self.labels[index]
      return one_x,one_y

    def __len__(self):
      return self.labels.shape[0]

train_ds = ToyDataset(X_train,y_train)
test_ds = ToyDataset(X_test,y_test)

In [25]:
len(train_ds)

5

In [46]:
torch.manual_seed(123)

train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0,
    drop_last=True
    )

In [47]:
test_Loader = DataLoader(
    dataset = test_ds,
    batch_size = 1,
    shuffle = False,
    num_workers =0
)

In [28]:
for idx,(x,y) in enumerate(train_loader):
  print(f"Batch {idx+1}:{x},{y}")

Batch 1:tensor([[ 2.3000, -1.1000, -1.1000],
        [-0.9000,  2.9000,  0.7000]]),tensor([1, 0])
Batch 2:tensor([[-1.2000,  3.1000,  1.2000],
        [-0.5000,  2.6000, -0.8000]]),tensor([0, 0])


In [32]:
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(3,2)
optimizer = torch.optim.SGD(model.parameters(),lr=0.5)
num_epochs = 3

for epoch in range(num_epochs):
  model.train()
  for batch_idx,(features,labels) in enumerate(train_loader):
    logits = model(features)
    loss = F.cross_entropy(logits,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch:{epoch+1:03d}/{num_epochs:03d}"
          f"| Batch{batch_idx+1:03d}/{len(train_loader):03d}"
          f"| Train loss :{loss:.2f}")

  model.eval()

Epoch:001/003| Batch001/002| Train loss :0.68
Epoch:001/003| Batch002/002| Train loss :0.51
Epoch:002/003| Batch001/002| Train loss :0.29
Epoch:002/003| Batch002/002| Train loss :0.19
Epoch:003/003| Batch001/002| Train loss :0.10
Epoch:003/003| Batch002/002| Train loss :0.02


In [40]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(params)

782


In [49]:
model.eval()
with torch.no_grad():
  outputs = model(X_train)
print(outputs)

tensor([[ 2.3768, -2.6285],
        [ 2.2207, -2.4441],
        [ 1.8829, -2.0872],
        [-2.1188,  1.6964],
        [-2.4648,  1.9449]])


In [51]:
predicted_labels = torch.argmax(outputs,dim=1)
print(predicted_labels)

tensor([0, 0, 0, 1, 1])


In [53]:
train_ds.labels

tensor([0, 0, 0, 1, 1])

In [54]:
predicted_labels == train_ds.labels

tensor([True, True, True, True, True])

In [55]:
def compute_accuracy(model,loader):
  correct = 0
  total = 0
  for idx,(features,labels) in enumerate(loader):
    with torch.no_grad():
      logits = model(features)
    predicted_labels = torch.argmax(logits,dim=1)
    total += labels.shape[0]
    correct += (predicted_labels == labels).sum().item()
  return correct/total


In [56]:
compute_accuracy(model,train_loader)

1.0

In [57]:
compute_accuracy(model,test_Loader)

1.0

In [58]:
torch.save(model.state_dict(),"model.pth")

In [59]:
model = NeuralNetwork(3,2)
model.load_state_dict(torch.load("model.pth"))

<ipython-input-59-a2574e1a72ac>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>